<div style="display: flex; align-items: center;">
    <h1>Optimizing parameters in a WOFOST crop model using <code>diffWOFOST</code></h1>
    <img src="https://raw.githubusercontent.com/WUR-AI/diffWOFOST/refs/heads/main/docs/logo/diffwofost.png" width="150" style="margin-left: 20px;">
</div>


This Jupyter notebook demonstrates the optimization of parameters in a
differentiable model using the `diffwofost` package. The package provides
differentiable implementations of the WOFOST model and its associated
sub-models. As `diffwofost` is under active development, this notebook focuses on
one sub-models: `phenology`. 

## 1. Phenology

In this section, we will demonstrate how to optimize the parameters `TSUMEM`, `TBASEM`, `TSUM1` and `TSUM2`in
phenology model using a differentiable version of phenology.
The optimization will be done using the Adam optimizer from `torch.optim`.

### 1.1 software requirements

To run this notebook, we need to install the `diffwofost`; the differentiable
version of WOFOST models. Since the package is constantly under development, make
sure you have the latest version of `diffwofost` installed in your
python environment. You can install it using pip:

In [ ]:
# install diffwofost
!pip install diffwofost

In [1]:
# ---- import libraries ----
import copy
import torch
import numpy
from pathlib import Path
from diffwofost.physical_models.utils import EngineTestHelper
from diffwofost.physical_models.utils import prepare_engine_input
from diffwofost.physical_models.utils import get_test_data

In [2]:
# ---- disable a warning: this will be fixed in the future ----
import warnings
warnings.filterwarnings("ignore", message="To copy construct from a tensor.*")

### 1.2. Data

A test dataset of `DVS` (Development stage) will be used to optimize the parameters:
- `TSUMEM`: Temperature sum from sowing to emergence,
- `TBASEM`: Base temperature for emergence,
- `TSUM1`: Temperature sum from emergence to anthesis,
- `TSUM2`: Temperature sum from anthesis to maturity. 

The data is stored in PCSE tests folder, and can be doewnloded from PCSE repsository.
You can select any of the files related to `phenology` model with a file name that follwos the pattern
`test_phenology_wofost72_*.yaml`. Each file contains different data depending on the locatin and crop type.
For example, you can download the file "test_phenology_wofost72_01.yaml" as:

In [3]:
import urllib.request

url = "https://raw.githubusercontent.com/ajwdewit/pcse/refs/heads/master/tests/test_data/test_phenology_wofost72_01.yaml"
filename = "test_phenology_wofost72_01.yaml"

urllib.request.urlretrieve(url, filename)
print(f"Downloaded: {filename}")

Downloaded: test_phenology_wofost72_01.yaml


We also need to download a config file to be able to run each crop module. This will change in the future versions. To donwload the config file, you can use the following command:

In [4]:
url = "https://raw.githubusercontent.com/WUR-AI/diffWOFOST/refs/heads/main/tests/physical_models/test_data/WOFOST_Phenology.conf"
filename = "WOFOST_Phenology.conf"

urllib.request.urlretrieve(url, filename)
print(f"Downloaded: {filename}")

Downloaded: WOFOST_Phenology.conf


In [5]:
# ---- Check the path to the files that are downloaded as explained above ----
test_data_path = "test_phenology_wofost72_01.yaml"
config_path = "WOFOST_Phenology.conf"

In [ ]:
# ---- Here we read the test data and set some variables ----
test_data = get_test_data(test_data_path)

crop_model_params = [
    "TSUMEM",
    "TBASEM",
    "TEFFMX",
    "TSUM1",
    "TSUM2",
    "IDSL",
    "DLO",
    "DLC",
    "DVSI",
    "DVSEND",
    "DTSMTB",
    "VERNSAT",
    "VERNBASE",
    "VERNDVS",
]
(crop_model_params_provider, weather_data_provider, agro_management_inputs, _) = (
    prepare_engine_input(test_data, crop_model_params)
)

expected_results = test_data["ModelResults"]
expected_dvs = torch.tensor([float(item["DVS"]) for item in expected_results], dtype=torch.float32
).unsqueeze(0) # shape: [1, time_steps]

# ---- dont change this: in this config file we specified the diffrentiable version of leaf_dynamics ----
config_path = str(Path(config_path).resolve()) 

### 1.3. Helper classes/functions

The model parameters shoudl stay in a valid range. To ensure this, we will use
`BoundedParameter` class with (min, max) and initial values for each
parameter. You might change these values depending on the crop type and
location. But dont use a very small range, otherwise gradiants will be very
small and the optimization will be very slow.

In [8]:
# ---- Adjust the values if needed  ----
TSUMEM_MIN, TSUMEM_MAX, TSUMEM_INIT = (0.0, 100, 10)
TBASEM_MIN, TBASEM_MAX, TBASEM_INIT = (0.0, 10.0, 1.0)
TSUM1_MIN, TSUM1_MAX, TSUM1_INIT = (0.0, 1000, 100)
TSUM2_MIN, TSUM2_MAX, TSUM2_INIT = (0.0, 2000, 500)

# ---- Helper for bounded parameters ----
class BoundedParameter(torch.nn.Module):
    def __init__(self, low, high, init_value):
        super().__init__()
        self.low = low
        self.high = high

        # Normalize to [0, 1]
        init_norm = (init_value - low) / (high - low)

        # Parameter in raw logit space
        self.raw = torch.nn.Parameter(torch.logit(torch.tensor(init_norm, dtype=torch.float32), eps=1e-6))

    def forward(self):
        return self.low + (self.high - self.low) * torch.sigmoid(self.raw)


Another helper class is `OptDiffPhenology` which is a subclass of `torch.nn.Module`. 
We use this class to wrap the `EngineTestHelper` function and make it easier to run the model `phenology`.

In [11]:
# ---- Wrap the model with torch.nn.Module----
class OptDiffPhenology(torch.nn.Module):
    def __init__(self, crop_model_params_provider, weather_data_provider, agro_management_inputs, config_path):
        super().__init__()
        self.crop_model_params_provider = crop_model_params_provider
        self.weather_data_provider = weather_data_provider
        self.agro_management_inputs = agro_management_inputs
        self.config_path = config_path

        # bounded parameters
        self.TSUMEM = BoundedParameter(TSUMEM_MIN, TSUMEM_MAX, TSUMEM_INIT)
        self.TBASEM = BoundedParameter(TBASEM_MIN, TBASEM_MAX, TBASEM_INIT)
        self.TSUM1 = BoundedParameter(TSUM1_MIN, TSUM1_MAX, TSUM1_INIT)
        self.TSUM2 = BoundedParameter(TSUM2_MIN, TSUM2_MAX, TSUM2_INIT)

    def forward(self):
        # currently, copying is needed due to an internal issue in engine
        crop_model_params_provider_ = copy.deepcopy(self.crop_model_params_provider)

        TSUMEM_val = self.TSUMEM()
        TBASEM_val = self.TBASEM()
        TSUM1_val = self.TSUM1()
        TSUM2_val = self.TSUM2()
        
        # pass new value of parameters to the model
        crop_model_params_provider_.set_override("TSUMEM", TSUMEM_val, check=False)
        crop_model_params_provider_.set_override("TBASEM", TBASEM_val, check=False)
        crop_model_params_provider_.set_override("TSUM1", TSUM1_val, check=False)
        crop_model_params_provider_.set_override("TSUM2", TSUM2_val, check=False)

        engine = EngineTestHelper(
            crop_model_params_provider_,
            self.weather_data_provider,
            self.agro_management_inputs,
            self.config_path,
        )
        engine.run_till_terminate()
        results = engine.get_output()
        
        return torch.stack([item["DVS"] for item in results]) # shape: [1, time_steps]

In [13]:
# ----  Create model ---- 
opt_model = OptDiffPhenology(
    crop_model_params_provider,
    weather_data_provider,
    agro_management_inputs,
    config_path,
)

In [14]:
# ----  Optimizer ---- 
optimizer = torch.optim.Adam(opt_model.parameters(), lr=0.1)

# ----  We use relative MAE as loss because there are two outputs with different untis ----  
denom = torch.mean(torch.abs(expected_dvs)) 

# Training loop (example)
for step in range(101):
    optimizer.zero_grad()
    results = opt_model() 
    mae = torch.mean(torch.abs(results - expected_dvs))
    loss = mae / denom  # example: relative mean absolute error
    loss.backward()
    optimizer.step()

    if step % 10 == 0:
        print(
            f"Step {step}, Loss {loss.item():.4f}, TSUMEM {opt_model.TSUMEM().item():.4f}, TBASEM {opt_model.TBASEM().item():.4f}, TSUM1 {opt_model.TSUM1().item():.4f}, TSUM2 {opt_model.TSUM2().item():.4f},"
        )

Step 0, Loss 0.2719, TSUMEM 10.9366, TBASEM 1.0936, TSUM1 109.3669, TSUM2 538.4286,
Step 10, Loss 0.1660, TSUMEM 24.9445, TBASEM 2.4692, TSUM1 254.1459, TSUM2 1000.2973,
Step 20, Loss 0.0194, TSUMEM 49.5210, TBASEM 4.8817, TSUM1 494.5926, TSUM2 1462.7181,
Step 30, Loss 0.0334, TSUMEM 60.4199, TBASEM 5.3454, TSUM1 517.5145, TSUM2 1470.7230,
Step 40, Loss 0.0238, TSUMEM 61.3564, TBASEM 4.9118, TSUM1 403.4216, TSUM2 1419.6570,
Step 50, Loss 0.0025, TSUMEM 61.8056, TBASEM 4.6627, TSUM1 409.8507, TSUM2 1605.2671,
Step 60, Loss 0.0010, TSUMEM 63.8338, TBASEM 4.6446, TSUM1 412.4301, TSUM2 1562.7145,
Step 70, Loss 0.0041, TSUMEM 65.6777, TBASEM 4.5972, TSUM1 422.9336, TSUM2 1601.4424,
Step 80, Loss 0.0045, TSUMEM 67.2317, TBASEM 4.5117, TSUM1 429.2060, TSUM2 1567.4720,
Step 90, Loss 0.0010, TSUMEM 68.6938, TBASEM 4.4138, TSUM1 417.7693, TSUM2 1582.8475,
Step 100, Loss 0.0038, TSUMEM 70.1681, TBASEM 4.3225, TSUM1 418.5266, TSUM2 1566.5803,


In [15]:
# ---- validate the results using test data ---- 
print(f"Actual TSUMEM {crop_model_params_provider["TSUMEM"].item():.4f}, TBASEM {crop_model_params_provider["TBASEM"].item():.4f}")
print(f"Actual TSUM1 {crop_model_params_provider["TSUM1"].item():.4f}, TSUM2 {crop_model_params_provider["TSUM2"].item():.4f}")

Actual TSUMEM 90.0000, TBASEM 3.0000
Actual TSUM1 418.0000, TSUM2 1578.0000
